# Détection d'Anomalies avec PyOD (FD001)

Ce notebook charge les données traitées et entraîne un modèle **Isolation Forest** pour détecter les anomalies potentiellement liées aux défaillances.

## Méthodologie
1. **Entraînement** : Sur le début de vie des moteurs (considéré comme sain).
2. **Évaluation** : Sur la fin de vie (considéré comme anormal).
3. **Métriques** : Precision, Recall, F1-Score, ROC-AUC.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from pyod.models.iforest import IForest
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, accuracy_score

# Config
PROCESSED_PATH = Path('../data/processed')

# Chargement
print("Chargement des données...")
train = pd.read_csv(PROCESSED_PATH / 'train_FD001_features.csv')
test = pd.read_csv(PROCESSED_PATH / 'test_FD001_features.csv')

# Feature Selection (les capteurs les plus pertinents pour FD001)
# On prend les features 'mean' des capteurs sensisbles
features = [c for c in train.columns if 'mean' in c]
print(f"Nombre de features utilisées: {len(features)}")

# --- Création des Labels (Proxy) ---
# Hypothèse : RUL > 50 = Normal (0), RUL <= 50 = Anomalie (1)
train['label'] = (train['RUL'] <= 50).astype(int)
print(train['label'].value_counts())

# --- Split Train/Test pour l'Anomaly Detection ---
# Train : Uniquement des données SAINES (RUL > 50) pour apprendre la normalité
X_train_normal = train[train['label'] == 0][features]

# Test : Mélange de données saines et anormales pour évaluer
X_eval = train[features]
y_eval = train['label']

# --- Modélisation (Isolation Forest) ---
print("Entraînement Isolation Forest...")
clf = IForest(contamination=0.1, random_state=42)
clf.fit(X_train_normal)

# Prédiction sur tout le dataset (Train)
y_pred = clf.predict(X_eval)  # 0: Inliers, 1: Outliers
y_scores = clf.decision_function(X_eval)

# --- Évaluation ---
print("\n--- Rapport de Classification ---")
print(classification_report(y_eval, y_pred, target_names=['Normal', 'Anomalie']))

roc = roc_auc_score(y_eval, y_scores)
print(f"ROC-AUC Score: {roc:.4f}")

# Courbe ROC
fpr, tpr, _ = roc_curve(y_eval, y_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Isolation Forest (AUC={roc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Anomaly Detection')
plt.legend()
plt.show()